# GenCoMo Tutorial: Cylinder Mesh Demo

This tutorial shows how to create and visualize simple cylindrical mesh geometries using GenCoMo. Cylinders are the most basic building blocks for representing neuronal processes like axons and dendrites.

## What You'll Learn

- Create cylinder meshes with different parameters
- Visualize cylinders in 3D with proper aspect ratios
- Explore cylinder cross-sections interactively
- Analyze mesh properties like volume and surface area

## Setup

First, let's import the necessary libraries:

In [1]:
import numpy as np
import trimesh
import plotly.graph_objects as go

# Import GenCoMo functions
from gencomo.demos import create_cylinder_mesh
from gencomo.mesh import (
    visualize_mesh_3d, 
    analyze_mesh_properties, 
    visualize_mesh_slice_interactive
)

print("✅ Libraries imported successfully!")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
✅ Libraries imported successfully!


## Creating Basic Cylinders

Let's start by creating a simple cylinder representing a neuronal process:

In [2]:
# Create a basic cylinder mesh
cylinder = create_cylinder_mesh(
    length=100.0,    # 100 micrometers long
    radius=5.0,      # 5 micrometers radius
    resolution=16,   # 16 sides for smooth appearance
    axis='z'         # oriented along z-axis
)

print(f"Created cylinder with {len(cylinder.vertices)} vertices and {len(cylinder.faces)} faces")
print(f"Volume: {cylinder.volume:.2f} μm³")
print(f"Surface area: {cylinder.area:.2f} μm²")

Created cylinder with 34 vertices and 64 faces
Volume: 7653.67 μm³
Surface area: 3274.52 μm²


## 3D Visualization

Let's visualize our cylinder in 3D with proper aspect ratios:

In [3]:
# Visualize the cylinder
fig = visualize_mesh_3d(
    cylinder, 
    title="Simple Cylinder Mesh",
    color="lightblue",
    backend="plotly"
)
fig.show()

## Interactive Cross-Section Exploration

Now let's explore the internal structure using interactive slicing:

In [4]:
# Interactive slice view of the cylinder
fig_cylinder_slice = visualize_mesh_slice_interactive(
    cylinder,
    title="Interactive Cylinder Cross-Sections",
    slice_color="darkblue",
    mesh_color="lightblue",
    mesh_opacity=0.3
)
fig_cylinder_slice.show()

## Exploring Different Parameters

Let's create cylinders with different dimensions to see how they affect the geometry:

In [5]:
# Create cylinders with different parameters
thin_cylinder = create_cylinder_mesh(length=80.0, radius=2.0, resolution=12)
thick_cylinder = create_cylinder_mesh(length=60.0, radius=8.0, resolution=20)
long_cylinder = create_cylinder_mesh(length=150.0, radius=3.0, resolution=16)

cylinders = {
    "Standard": cylinder,
    "Thin": thin_cylinder,
    "Thick": thick_cylinder,
    "Long": long_cylinder
}

# Analyze each cylinder
for name, cyl in cylinders.items():
    props = analyze_mesh_properties(cyl)
    print(f"\n=== {name} Cylinder ===")
    print(f"Vertices: {props['num_vertices']}, Faces: {props['num_faces']}")
    print(f"Volume: {props['volume']:.2f} μm³")
    print(f"Surface Area: {props['surface_area']:.2f} μm²")
    print(f"Length (Z): {props['bounds']['z_range'][1] - props['bounds']['z_range'][0]:.1f} μm")


=== Standard Cylinder ===
Vertices: 34, Faces: 64
Volume: 7653.67 μm³
Surface Area: 3274.52 μm²
Length (Z): 100.0 μm

=== Thin Cylinder ===
Vertices: 26, Faces: 48
Volume: 960.00 μm³
Surface Area: 1017.87 μm²
Length (Z): 80.0 μm

=== Thick Cylinder ===
Vertices: 42, Faces: 80
Volume: 11866.25 μm³
Surface Area: 3399.08 μm²
Length (Z): 60.0 μm

=== Long Cylinder ===
Vertices: 34, Faces: 64
Volume: 4132.98 μm³
Surface Area: 2864.41 μm²
Length (Z): 150.0 μm


## Comparison Visualization

Let's compare all our cylinders side by side:

In [6]:
# Create a comparison plot
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "scatter3d"}, {"type": "scatter3d"}],
           [{"type": "scatter3d"}, {"type": "scatter3d"}]],
    subplot_titles=["Standard", "Thin", "Thick", "Long"]
)

colors = ["lightblue", "lightgreen", "orange", "purple"]
positions = [(1, 1), (1, 2), (2, 1), (2, 2)]

for i, (name, cyl) in enumerate(cylinders.items()):
    vertices = cyl.vertices
    faces = cyl.faces
    
    trace = go.Mesh3d(
        x=vertices[:, 0],
        y=vertices[:, 1],
        z=vertices[:, 2],
        i=faces[:, 0],
        j=faces[:, 1],
        k=faces[:, 2],
        color=colors[i],
        opacity=0.8
    )
    
    fig.add_trace(trace, row=positions[i][0], col=positions[i][1])

fig.update_layout(
    title="Cylinder Variations Comparison",
    height=800,
    showlegend=False
)

fig.show()

## Resolution Effects

Let's see how the resolution parameter affects the mesh quality:

In [7]:
# Create cylinders with different resolutions
low_res = create_cylinder_mesh(length=50.0, radius=5.0, resolution=6)
med_res = create_cylinder_mesh(length=50.0, radius=5.0, resolution=12)
high_res = create_cylinder_mesh(length=50.0, radius=5.0, resolution=24)

resolutions = {
    "Low (6 sides)": low_res,
    "Medium (12 sides)": med_res,
    "High (24 sides)": high_res
}

for name, cyl in resolutions.items():
    props = analyze_mesh_properties(cyl)
    print(f"\n{name}:")
    print(f"  Vertices: {props['num_vertices']}, Faces: {props['num_faces']}")
    print(f"  Surface Area: {props['surface_area']:.2f} μm²")


Low (6 sides):
  Vertices: 14, Faces: 24
  Surface Area: 1629.90 μm²

Medium (12 sides):
  Vertices: 26, Faces: 48
  Surface Area: 1702.91 μm²

High (24 sides):
  Vertices: 50, Faces: 96
  Surface Area: 1721.61 μm²


## Saving Your Cylinder

Finally, let's save our cylinder mesh for later use:

In [8]:
# Save the cylinder mesh
import os

output_dir = "data/mesh"
os.makedirs(output_dir, exist_ok=True)

filename = f"{output_dir}/cylinder_demo.stl"
cylinder.export(filename)
print(f"💾 Saved cylinder mesh to: {filename}")

# Also save the variations
for name, cyl in cylinders.items():
    filename = f"{output_dir}/cylinder_{name.lower()}.stl"
    cyl.export(filename)
    print(f"💾 Saved {name.lower()} cylinder to: {filename}")

💾 Saved cylinder mesh to: data/mesh/cylinder_demo.stl
💾 Saved standard cylinder to: data/mesh/cylinder_standard.stl
💾 Saved thin cylinder to: data/mesh/cylinder_thin.stl
💾 Saved thick cylinder to: data/mesh/cylinder_thick.stl
💾 Saved long cylinder to: data/mesh/cylinder_long.stl


## Summary

In this tutorial, you learned how to:

1. **Create cylinder meshes** with custom length, radius, and resolution
2. **Visualize cylinders in 3D** with proper aspect ratios
3. **Explore cross-sections** using interactive slicing
4. **Compare different parameters** and their effects on geometry
5. **Analyze mesh properties** like volume and surface area
6. **Save meshes** to files for later use

## Next Steps

- Try the **Torus Demo** (01b_torus_demo.ipynb) for ring-like structures
- Explore **Branching Demo** (01c_branching_demo.ipynb) for Y-shaped geometries
- Learn about **Complete Neurons** (01d_neuron_demo.ipynb) with soma, dendrites, and axon
- Experiment with different cylinder orientations using the `axis` parameter
- Combine multiple cylinders to create more complex structures